## 如果是在Google Colab上執行，請先執行以下程式碼

In [ ]:
# from google.colab import userdata

# ngrok_key = userdata.get('ngrok_key')
# line_access_token = userdata.get('line_access_token')
# line_secret = userdata.get('line_secret')
# port = 5000

## 如果在本機執行，需先設定好`conda vars`變數，以取得各金鑰

### 程式說明
 - 從`conda`虛擬環境中取得line secret和line channel access token
 - 指定通訊埠為5000

In [ ]:
import os
line_access_token = os.environ.get('LINE_ACCESS_TOKEN')
line_secret = os.environ.get('LINE_SECRET')
port = 5000

### 載入套件
- `WebhookHandler`類別用來處理送入LINE Bot的訊息
- `InvalidSignatureError`例外處理用來處理簽名錯誤
- `Flask`用來建立網頁伺服器
- `request`用來處理HTTP請求
- `Configuration`用來設定LINE Bot的基本資訊
- `ApiClient`用來建立與LINE Bot的連線
- `MessagingApi`用來發送訊息
- `TextMessage`為文字類別物件
- `ReplyMessageRequest`為回覆訊息的請求物件

In [ ]:
from flask import Flask, request, abort

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    ReplyMessageRequest,
    TextMessage,    
)

from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
)


### 程式執行本體

In [ ]:
# 定義Flask應用程式起點

app = Flask(__name__)

# 設定LINE Bot的Webhook，指定LINE Bot的Channel Secret與Channel Access Token
# 指定處理Webhook的handler

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)

# 以下為正式處理的程式碼
# 指定網址，並且指定Webhook規定使用的POST方法而非GET方法
# 此處的處理程式可以自行命名，這邊使用慣例的callback
# 這邊處理的網址為根網址，即"/"。如果要指定其他網址，可以在"/"後面加上其他字串，如"/callback"
# 如果指定了其它字串，在LINE的設定中的Webhook也要隨之修改

@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    
    # 將讀取來的本體寫入記錄檔
    app.logger.info("Request body: " + body)

    # handle webhook body，如果簽名有問題，就直接中止並送出400錯訊訊息
    # 如果沒有問題，就直接丟給handler處理
    
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)
    # 這邊一定要送出"OK"，這是LINE規定的
    return 'OK'

if __name__ == "__main__":
    app.run(port=port)

In [ ]:
app = Flask(__name__)

configuration = Configuration(access_token=line_access_token)
handler = WebhookHandler(line_secret)


@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']

    body = request.get_data(as_text=True)
    print("BODY: ", body)
    
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'

# 加上處理訊息的部分
# 如果是「訊息事件」，且訊息是文字訊息的處理方法
# 先印出訊息本身的全部
# 透過ApiClient建立一個MessagingApi的物件
# 透過MessagingApi的reply_message_with_http_info方法，回覆訊息
# 這邊的訊息是一個文字訊息，內容是原本的訊息內容兩次

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    print("EVENT", event)
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)
        line_bot_api.reply_message_with_http_info(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[TextMessage(text=event.message.text),
                            TextMessage(text=event.message.text)]
            )
        )

# 程式開始執行

if __name__ == "__main__":
    app.run(port=port)